In [1]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator, SimulationManager
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
from nanowire.optics.utils.geometry import get_mask
import scipy.constants as consts
import scipy.integrate as intg
import IPython.display as disp
import matplotlib.pyplot as plt
%load_ext autoreload 
%autoreload 2
%matplotlib notebook

In [2]:
conf = Config('AbsorptionByRegion.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
sim.get_fluxes()
sim.get_field()
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

TypeError: unorderable types: bool() < str()

In [ ]:
s = Simulation(simulator=sim)

In [ ]:
abs_dict = s.absorption_per_layer()
print(abs_dict)

In [ ]:
total_absorbed = sum(dlist[1].real for layer, dlist in abs_dict.items())
print(total_absorbed)

In [ ]:
base_unit = s.conf['Simulation']['base_unit']
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]
freq = s.conf[('Simulation', 'params', 'frequency', 'value')]

In [ ]:
abs_ito = abs_dict['ITO'][1]
abs_subs = abs_dict['Substrate'][1]
print("ITO Absorption: {}".format(abs_ito))
print("Substrate Absorption: {}".format(abs_subs))

In [ ]:
nw_alshell = s.layers['NW_AlShell']
xcoords = np.arange(0, s.period, s.dx)
ycoords = np.arange(0, s.period, s.dy)
outer_mask = get_mask(nw_alshell.shapes['shell'][0], xcoords, ycoords)
cyc_mask = np.ones_like(outer_mask) - outer_mask
inner_mask = get_mask(nw_alshell.shapes['core'][0], xcoords, ycoords)
shell_mask = outer_mask - inner_mask
n_mat, k_mat = nw_alshell.get_nk_matrix(freq)

In [ ]:
plt.matshow(inner_mask)

In [ ]:
plt.matshow(shell_mask)

In [ ]:
plt.matshow(cyc_mask)

In [ ]:
Esqnk = s.normEsquared()*n_mat*k_mat

In [ ]:
s.data['normEsquarednk'] = Esqnk

In [ ]:
core_abs = 2*np.pi*freq*consts.epsilon_0*base_unit*s.integrate_quantity('normEsquarednk', mask=inner_mask, layer='NW_AlShell')
shell_abs = 2*np.pi*freq*consts.epsilon_0*base_unit*s.integrate_quantity('normEsquarednk', mask=shell_mask, layer='NW_AlShell')
cyc_abs = 2*np.pi*freq*consts.epsilon_0*base_unit*s.integrate_quantity('normEsquarednk', mask=cyc_mask, layer='NW_AlShell')

In [ ]:
print("Total NW Absorption: {}".format(abs_dict["NW_AlShell"][1]))
print("Core Absorption: {}".format(core_abs))
print("Shell Absorption: {}".format(shell_abs))
print("Cyclotene Absorption: {}".format(cyc_abs))

In [3]:
def integrate_regions(sim):
    base_unit = sim.conf['Simulation']['base_unit']
    Zo = consts.physical_constants['characteristic impedance of vacuum'][0]
    freq = sim.conf[('Simulation', 'params', 'frequency', 'value')]
    nw_alshell = sim.layers['NW_AlShell']
    xcoords = np.arange(0, sim.period, sim.dx)
    ycoords = np.arange(0, sim.period, sim.dy)
    outer_mask = get_mask(nw_alshell.shapes['shell'][0], xcoords, ycoords)
    cyc_mask = np.ones_like(outer_mask) - outer_mask
    inner_mask = get_mask(nw_alshell.shapes['core'][0], xcoords, ycoords)
    shell_mask = outer_mask - inner_mask
    n_mat, k_mat = nw_alshell.get_nk_matrix(freq)
    Esqnk = sim.normEsquared()*n_mat*k_mat
    sim.data['normEsquarednk'] = Esqnk
    core_abs = 2*np.pi*freq*consts.epsilon_0*base_unit*sim.integrate_quantity('normEsquarednk', mask=inner_mask, layer='NW_AlShell')
    shell_abs = 2*np.pi*freq*consts.epsilon_0*base_unit*sim.integrate_quantity('normEsquarednk', mask=shell_mask, layer='NW_AlShell')
    cyc_abs = 2*np.pi*freq*consts.epsilon_0*base_unit*sim.integrate_quantity('normEsquarednk', mask=cyc_mask, layer='NW_AlShell')
    return core_abs, shell_abs, cyc_abs

# Frequency Sweep 

In [2]:
manager = SimulationManager('AbsorptionByRegionFreqSweep.yml')
manager.make_confs()
manager.execute_jobs()

01/05/2018 11:11:20 AM [nanowire.optics.simulate:INFO] - Constructing simulator objects ...
01/05/2018 11:11:20 AM [nanowire.optics.simulate:INFO] - Executing sims in parallel using 6 cores ...
01/05/2018 11:11:20 AM [nanowire.optics.simulate:INFO] - Executing sim a25fabc18f
01/05/2018 11:11:20 AM [nanowire.optics.simulate:INFO] - Executing sim e7d0cb0ba3
01/05/2018 11:11:20 AM [nanowire.optics.simulate:INFO] - Executing sim 31a611de0d
01/05/2018 11:11:20 AM [nanowire.optics.simulate:INFO] - Executing sim c16492fe9e
01/05/2018 11:11:20 AM [nanowire.optics.simulate:INFO] - Executing sim e8bda3a515
01/05/2018 11:11:20 AM [nanowire.optics.simulate:INFO] - Executing sim af7cb156db
01/05/2018 11:15:42 AM [nanowire.optics.simulate:INFO] - Simulation 31a611de0d completed in 261.60 seconds!
01/05/2018 11:15:42 AM [nanowire.optics.simulate:INFO] - Executing sim 1d53534095
01/05/2018 11:15:42 AM [nanowire.optics.simulate:INFO] - 39 out of 40 simulations remaining
01/05/2018 11:15:44 AM [nanowire

In [4]:
for conf in manager.:
    print(conf.id)

AttributeError: 'Config' object has no attribute 'id'

In [6]:
lin = np.linspace(0, .25, 5)
dx = lin[1] - lin[0]
aran = np.arange(0, .25, dx)
print(lin)
print(aran)

[ 0.     0.062  0.125  0.188  0.25 ]
[ 0.     0.062  0.125  0.188]


In [8]:
period = .25
num_samples = 5
dx = period / num_samples
aran = np.arange(0, .25, dx)
lin = np.linspace(0, period, num_samples)
lin = np.linspace(0, period, num_samples, endpoint=False)
print(aran)
print(lin)

[ 0.    0.05  0.1   0.15  0.2 ]
[ 0.    0.05  0.1   0.15  0.2 ]
